<a href="https://colab.research.google.com/github/Kush-Singh-26/Image_Caption/blob/main/CaptionColab_2_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training Continued

In [ ]:
import torch
import torchvision.transforms as transforms
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os
import nltk
import json
import collections
from collections import Counter
import random
import time
from torch.nn.utils.rnn import pad_sequence
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [ ]:
!mkdir -p /content/coco/images/train2014
!mkdir -p /content/coco/images/val2014
!mkdir -p /content/coco/annotations


In [ ]:
!wget http://images.cocodataset.org/zips/train2014.zip -P /content/coco/images/
!wget http://images.cocodataset.org/zips/val2014.zip -P /content/coco/images/


--2025-05-05 03:53:21--  http://images.cocodataset.org/zips/train2014.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.217.192.209, 52.217.118.89, 52.216.59.185, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|52.217.192.209|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13510573713 (13G) [application/zip]
Saving to: ‘/content/coco/images/train2014.zip’

train2014.zip       100%[===================>]  12.58G  50.9MB/s    in 4m 21s  

2025-05-05 03:57:42 (49.3 MB/s) - ‘/content/coco/images/train2014.zip’ saved [13510573713/13510573713]

--2025-05-05 03:57:42--  http://images.cocodataset.org/zips/val2014.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 16.15.177.212, 16.15.194.211, 52.217.229.249, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|16.15.177.212|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6645013297 (6.2G) [application/zip]
Saving to: ‘/content/coco/i

In [ ]:
!wget http://images.cocodataset.org/annotations/annotations_trainval2014.zip -P /content/coco/annotations/


--2025-05-05 03:59:45--  http://images.cocodataset.org/annotations/annotations_trainval2014.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.217.124.233, 16.15.184.101, 52.217.85.132, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|52.217.124.233|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252872794 (241M) [application/zip]
Saving to: ‘/content/coco/annotations/annotations_trainval2014.zip’

annotations_trainva 100%[===================>] 241.16M  58.1MB/s    in 4.6s    

2025-05-05 03:59:50 (51.9 MB/s) - ‘/content/coco/annotations/annotations_trainval2014.zip’ saved [252872794/252872794]



In [ ]:
!unzip -q /content/coco/images/train2014.zip -d /content/coco/images/
!unzip -q /content/coco/images/val2014.zip -d /content/coco/images/
!unzip -q /content/coco/annotations/annotations_trainval2014.zip -d /content/coco/annotations/


In [ ]:
!wget -c "https://github.com/Delphboy/karpathy-splits/raw/main/dataset_coco.json?download=" -O /content/dataset_coco.json


--2025-05-05 04:03:44--  https://github.com/Delphboy/karpathy-splits/raw/main/dataset_coco.json?download=
Resolving github.com (github.com)... 140.82.116.3
Connecting to github.com (github.com)|140.82.116.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/Delphboy/karpathy-splits/main/dataset_coco.json?download=true [following]
--2025-05-05 04:03:44--  https://media.githubusercontent.com/media/Delphboy/karpathy-splits/main/dataset_coco.json?download=true
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 408860810 (390M) [application/octet-stream]
Saving to: ‘/content/dataset_coco.json’

/content/dataset_co 100%[===================>] 389.92M  71.5MB/s    in 3.8s    

2025-05-05 04:0

In [ ]:
import json

with open("/content/dataset_coco.json", "r") as f:
    karpathy_data = json.load(f)

karpathy_images = karpathy_data['images']

train_data = [img for img in karpathy_images if img['split'] == 'train']
val_data = [img for img in karpathy_images if img['split'] == 'val']
test_data = [img for img in karpathy_images if img['split'] == 'test']

print(f"# Train: {len(train_data)} | Val: {len(val_data)} | Test: {len(test_data)}")


# Train: 82783 | Val: 5000 | Test: 5000


In [ ]:
import collections
from collections import Counter
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
class Vocabulary:
    def __init__(self, freq_threshold=5):
        self.freq_threshold = freq_threshold
        # Initialize with special tokens
        self.word2idx = {"<pad>": 0, "<start>": 1, "<end>": 2, "<unk>": 3}
        self.idx2word = {0: "<pad>", 1: "<start>", 2: "<end>", 3: "<unk>"}
        self.idx = 4 # Next index to assign

    def build_vocabulary(self, sentence_list):
        frequencies = Counter()
        print(f"Building vocabulary from {len(sentence_list)} sentences...")
        for i, sentence in enumerate(sentence_list):
            tokens = nltk.tokenize.word_tokenize(sentence.lower())
            frequencies.update(tokens)
            if (i+1) % 100000 == 0:
                 print(f"Processed {i+1}/{len(sentence_list)} sentences")


        original_size = len(frequencies)
        filtered_freq = {word: freq for word, freq in frequencies.items() if freq >= self.freq_threshold}

        for word in filtered_freq:
            if word not in self.word2idx:
                self.word2idx[word] = self.idx
                self.idx2word[self.idx] = word
                self.idx += 1
        print(f"Original vocab size: {original_size}, Filtered size (freq>={self.freq_threshold}): {len(self.word2idx)}")


    def numericalize(self, text):
        tokens = nltk.tokenize.word_tokenize(text.lower())
        return [self.word2idx.get(token, self.word2idx["<unk>"]) for token in tokens]

    def __len__(self):
        return self.idx

In [ ]:
import pickle
import os

class Vocabulary:
    def __init__(self, freq_threshold=5):
        self.freq_threshold = freq_threshold
        self.word2idx = {"<pad>": 0, "<start>": 1, "<end>": 2, "<unk>": 3}
        self.idx2word = {0: "<pad>", 1: "<start>", 2: "<end>", 3: "<unk>"}
        self.idx = 4
    def build_vocabulary(self, sentence_list):
        frequencies = Counter()
        for sentence in sentence_list:
            tokens = nltk.tokenize.word_tokenize(sentence.lower())
            frequencies.update(tokens)
        filtered_freq = {word: freq for word, freq in frequencies.items() if freq >= self.freq_threshold}
        for word in filtered_freq:
            if word not in self.word2idx:
                self.word2idx[word] = self.idx
                self.idx2word[self.idx] = word
                self.idx += 1
    def numericalize(self, text):
        tokens = nltk.tokenize.word_tokenize(text.lower())
        return [self.word2idx.get(token, self.word2idx["<unk>"]) for token in tokens]
    def __len__(self):
        return self.idx

vocab_pickle_path = '/content/vocab.pkl'


vocab = None
print(f"\n--- Attempting to load Vocabulary from Pickle: {vocab_pickle_path} ---")

if not os.path.exists(vocab_pickle_path):
    print(f"Error: Vocabulary pickle file not found at {vocab_pickle_path}")
    exit()
else:
    try:
        # Open the file in 'rb' (read binary) mode
        with open(vocab_pickle_path, 'rb') as f:
            # Load the object from the file
            vocab = pickle.load(f)

        if isinstance(vocab, Vocabulary):
            vocab_size = len(vocab)
            print(f"Vocabulary loaded successfully. Size: {vocab_size}")

        else:
            print(f"Error: Loaded object is not an instance of the Vocabulary class!")
            vocab = None # Reset vocab if loading failed verification
            exit() # Exit if the correct object wasn't loaded

    except FileNotFoundError:
        print(f"Error: Vocabulary pickle file not found at {vocab_pickle_path}")
        exit()
    except pickle.UnpicklingError:
        print(f"Error: Could not unpickle the file. It might be corrupted or saved with a different protocol.")
        exit()
    except Exception as e:
        print(f"An unexpected error occurred while loading the vocabulary pickle file: {e}")
        exit()

if vocab:
    print("\nVocabulary object is ready to use.")

else:
    print("\nFailed to load or initialize vocabulary. Cannot proceed.")


--- Attempting to load Vocabulary from Pickle: /content/vocab.pkl ---
Vocabulary loaded successfully. Size: 8853

Vocabulary object is ready to use.


In [ ]:
class EncoderCNN(nn.Module):
  def __init__(self, embed_size, dropout_p=0.5, fine_tune=True): # Added fine_tune flag
    super(EncoderCNN, self).__init__()
    print(f"Initializing EncoderCNN: embed_size={embed_size}, dropout={dropout_p}, fine_tune={fine_tune}")
    resnet = models.resnet101(weights=models.ResNet101_Weights.IMAGENET1K_V1) # Use updated weights call

    # Freeze all layers initially
    for param in resnet.parameters():
      param.requires_grad = False

    if fine_tune:
      print("Fine-tuning ResNet: Unfreezing layer4 parameters.")
      for param in resnet.layer4.parameters(): # Unfreeze layer 4
          param.requires_grad = True

    self.resnet = nn.Sequential(*list(resnet.children())[:-1])

    # Add trainable layers
    self.fc = nn.Linear(resnet.fc.in_features, embed_size)
    self.bn = nn.BatchNorm1d(embed_size, momentum=0.01) # BatchNorm after FC
    self.dropout = nn.Dropout(dropout_p) # Dropout layer

    # Initialize weights for the new layers
    self.fc.weight.data.normal_(0.0, 0.02)
    self.fc.bias.data.fill_(0)

  def forward(self, images):
    with torch.no_grad() if not self.training else torch.enable_grad():
        features = self.resnet(images) # [B, C, 1, 1]

    features = features.squeeze(3).squeeze(2) # [B, C]
    features = self.fc(features)              # [B, E]
    features = self.bn(features)              # [B, E] - Apply BN before dropout
    features = self.dropout(features)         # [B, E] - Apply dropout
    return features


In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, embed_size, hidden_size, vocab_size, num_layers=1, dropout_p=0.5):
        super().__init__()
        print(f"Initializing DecoderRNN: embed_size={embed_size}, hidden_size={hidden_size}, vocab_size={vocab_size}, num_layers={num_layers}, dropout={dropout_p}")
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.embed_dropout = nn.Dropout(dropout_p) # Dropout after embedding
        # Apply LSTM dropout between layers only if num_layers > 1
        lstm_dropout = dropout_p if num_layers > 1 else 0
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True, dropout=lstm_dropout)
        self.dropout = nn.Dropout(dropout_p) # Dropout before final linear layer
        self.linear = nn.Linear(hidden_size, vocab_size)

        # Layers to initialize LSTM state from image features
        self.init_h = nn.Linear(embed_size, hidden_size)
        self.init_c = nn.Linear(embed_size, hidden_size)

        # Initialize weights
        self.embed.weight.data.uniform_(-0.1, 0.1)
        self.linear.weight.data.uniform_(-0.1, 0.1)
        self.linear.bias.data.fill_(0)
        self.init_h.weight.data.uniform_(-0.1, 0.1)
        self.init_h.bias.data.fill_(0)
        self.init_c.weight.data.uniform_(-0.1, 0.1)
        self.init_c.bias.data.fill_(0)


    def forward(self, features, captions):
        # features: [B, E], captions: [B, T] (T = sequence length)

        # Prepare initial LSTM state from image features
        # Need shape [num_layers, B, H]
        h0 = self.init_h(features).unsqueeze(0) # [1, B, H]
        c0 = self.init_c(features).unsqueeze(0) # [1, B, H]
        # If num_layers > 1, repeat the initial state for each layer
        if self.lstm.num_layers > 1:
             h0 = h0.repeat(self.lstm.num_layers, 1, 1)
             c0 = c0.repeat(self.lstm.num_layers, 1, 1)

        # Embed captions and apply dropout
        embeddings = self.embed(captions)    # [B, T, E]
        embeddings = self.embed_dropout(embeddings) # Apply dropout

        # Pass through LSTM
        # embeddings shape needs to be [B, T, E] for batch_first=True
        hiddens, _ = self.lstm(embeddings, (h0, c0))  # [B, T, H]

        # Apply dropout before the final linear layer
        hiddens = self.dropout(hiddens) # Apply dropout

        # Generate outputs (logits over vocabulary)
        outputs = self.linear(hiddens)  # [B, T, Vocab]
        return outputs


In [ ]:
class CocoDataset(Dataset):
    def __init__(self, data, img_root, vocab, transform=None):
        self.data = data
        self.img_root = img_root
        self.vocab = vocab
        self.transform = transform
        print(f"Initialized CocoDataset with {len(self.data)} items. Root: {self.img_root}")


    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        entry = self.data[idx]

        # Randomly select one caption
        caption_entry = random.choice(entry['sentences'])
        caption = caption_entry['raw']

        # Numericalize caption
        tokens = [self.vocab.word2idx["<start>"]] + \
                 self.vocab.numericalize(caption) + \
                 [self.vocab.word2idx["<end>"]]

        # Load image
        split_folder = entry.get('filepath', '')

        if split_folder in self.img_root:
             img_path = os.path.join(self.img_root, entry['filename'])
        else:
             img_path = os.path.join(self.img_root, split_folder, entry['filename'])

        try:
            image = Image.open(img_path).convert("RGB")
        except FileNotFoundError:
             print(f"Warning: Image not found at {img_path}. Skipping.")

             return self.__getitem__(0)


        # Apply transformations
        if self.transform:
            image = self.transform(image)

        return image, torch.Tensor(tokens).long() # Ensure tokens are LongTensor


In [ ]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

# Training Transform (with augmentation)
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)), # Random crop and resize
    transforms.RandomHorizontalFlip(),      # Random horizontal flip
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1), # Color augmentation
    transforms.ToTensor(),
    normalize
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)), # Resize to fixed size
    transforms.ToTensor(),
    normalize
])

In [ ]:
def collate_fn(batch):
    # Separate images and captions
    images, captions = zip(*batch)

    # Stack images (they should all be the same size)
    images = torch.stack(images, 0)

    # Pad captions to the max length in the batch
    # Make sure captions are Tensors before passing to pad_sequence
    captions = pad_sequence(captions, batch_first=True, padding_value=vocab.word2idx["<pad>"])

    return images, captions

In [ ]:
checkpoint_path = '/content/best_model_improved.pth'
num_epochs = 15 # Total desired epochs (including previous ones)

# Load hyperparameters from checkpoint if available, otherwise use defaults
print(f"--- Loading Checkpoint: {checkpoint_path} ---")
if not os.path.exists(checkpoint_path):
    print(f"Error: Checkpoint file not found at {checkpoint_path}")
    exit()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
checkpoint = torch.load(checkpoint_path, map_location=device, weights_only=False)
print(f"Checkpoint loaded successfully from device {checkpoint.get('device', 'unknown') if isinstance(checkpoint, dict) else 'unknown'}") # Check device if saved

# Extract parameters from checkpoint (provide defaults if keys might be missing)
embed_size = checkpoint.get('embed_size', 256)
hidden_size = checkpoint.get('hidden_size', 512)
num_layers = checkpoint.get('num_layers', 1)
dropout_prob = checkpoint.get('dropout_prob', 0.5)
fine_tune_encoder = checkpoint.get('fine_tune_encoder', True)
learning_rate = 3e-4   # Base LR - Can be loaded from optimizer state later
fine_tune_lr = 1e-5    # Fine-tune LR - Can be loaded from optimizer state later
weight_decay = 1e-5    # Weight decay
patience_early_stop = 3 # Needs to match the EarlyStopping instance setup
delta_early_stop = 0.005
patience_scheduler = 1

batch_size = 64        # Batch size (keep consistent or adjust based on memory)
num_workers = 2 if device.type == 'cuda' else 0 # Dataloader workers

print("\n--- Using Parameters ---")
print(f"  Embed Size: {embed_size}")
print(f"  Hidden Size: {hidden_size}")
print(f"  Num Layers (LSTM): {num_layers}")
print(f"  Dropout: {dropout_prob}")
print(f"  Fine-tune Encoder: {fine_tune_encoder}")
print(f"  Target Num Epochs: {num_epochs}")
print(f"  Device: {device}")


# Load Vocabulary
print("\n--- Loading Vocabulary ---")
if 'vocab' not in checkpoint:
    print("Error: Vocabulary not found in checkpoint. Cannot resume precisely.")
    exit()
vocab = checkpoint['vocab']
vocab_size = len(vocab)
print(f"Vocabulary loaded. Size: {vocab_size}")

--- Loading Checkpoint: /content/best_model_improved.pth ---
Checkpoint loaded successfully from device unknown

--- Using Parameters ---
  Embed Size: 256
  Hidden Size: 512
  Num Layers (LSTM): 1
  Dropout: 0.5
  Fine-tune Encoder: True
  Target Num Epochs: 15
  Device: cuda

--- Loading Vocabulary ---
Vocabulary loaded. Size: 8853


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# --- Instantiate Datasets and Dataloaders ---
train_img_root = '/content/coco/images/' # Base directory containing train2014, val2014 etc.
val_img_root = '/content/coco/images/'

train_dataset = CocoDataset(train_data, train_img_root, vocab, train_transform)
val_dataset = CocoDataset(val_data, val_img_root, vocab, val_transform)

num_workers = 2 if device.type == 'cuda' else 0
print(f"Using {num_workers} workers for DataLoaders.")

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn, num_workers=num_workers, pin_memory=True if device.type == 'cuda' else False)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn, num_workers=num_workers, pin_memory=True if device.type == 'cuda' else False)


Using device: cuda
Initialized CocoDataset with 82783 items. Root: /content/coco/images/
Initialized CocoDataset with 5000 items. Root: /content/coco/images/
Using 2 workers for DataLoaders.


In [ ]:
print("--- Initializing Models, Optimizer, etc. ---")
encoder = EncoderCNN(embed_size, dropout_p=dropout_prob, fine_tune=fine_tune_encoder).to(device)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size, num_layers, dropout_p=dropout_prob).to(device)

decoder_params = list(decoder.parameters())
encoder_fc_params = list(encoder.fc.parameters()) + list(encoder.bn.parameters())
encoder_finetune_params = []
if fine_tune_encoder:
    layer4_index = 7
    encoder_finetune_params = list(encoder.resnet[layer4_index].parameters())
    print(f"Optimizing {len(encoder_finetune_params)} parameter tensors from ResNet layer4 with LR {fine_tune_lr}")


# Base parameters (Decoder + Encoder FC/BN layers)
params_to_optimize = [
    {'params': decoder_params},
    {'params': encoder_fc_params}
]

if encoder_finetune_params:
    params_to_optimize.append({'params': encoder_finetune_params, 'lr': fine_tune_lr})

# Initialize optimizer
optimizer = optim.Adam(params_to_optimize, lr=learning_rate, weight_decay=weight_decay)

# Learning Rate Scheduler
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=patience_scheduler, verbose=True)

# Loss Function (ignore padding)
criterion = nn.CrossEntropyLoss(ignore_index=vocab.word2idx["<pad>"])


--- Initializing Models, Optimizer, etc. ---
Initializing EncoderCNN: embed_size=256, dropout=0.5, fine_tune=True


Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:01<00:00, 152MB/s]


Fine-tuning ResNet: Unfreezing layer4 parameters.
Initializing DecoderRNN: embed_size=256, hidden_size=512, vocab_size=8853, num_layers=1, dropout=0.5
Optimizing 30 parameter tensors from ResNet layer4 with LR 1e-05


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [ ]:
class EarlyStopping:
    def __init__(self, patience=3, delta=0.0):
        self.patience = patience
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
        self.delta = delta # Minimum change to qualify as an improvement
        print(f"Initialized EarlyStopping: patience={patience}, delta={delta}")


    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
            print(f"EarlyStopping: Initial best loss set to {val_loss:.4f}")
        # Check if val_loss has improved significantly
        elif val_loss < self.best_loss - self.delta:
             print(f"EarlyStopping: Validation loss improved ({self.best_loss:.4f} --> {val_loss:.4f}). Resetting counter.")
             self.best_loss = val_loss
             self.counter = 0
        else:
            self.counter += 1
            print(f"EarlyStopping: No significant improvement for {self.counter}/{self.patience} epochs.")
            if self.counter >= self.patience:
                print("EarlyStopping: Triggering early stop.")
                self.early_stop = True

early_stopper = EarlyStopping(patience=patience_early_stop, delta=delta_early_stop)

Initialized EarlyStopping: patience=3, delta=0.005


In [ ]:
print("\n--- Loading State Dictionaries ---")
encoder.load_state_dict(checkpoint['encoder_state_dict'])
decoder.load_state_dict(checkpoint['decoder_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

# Load scheduler state if it was saved
if 'scheduler_state_dict' in checkpoint:
     try:
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        print("Scheduler state loaded.")
     except Exception as e:
         print(f"Warning: Could not load scheduler state: {e}")
else:
     print("Scheduler state not found in checkpoint, using new scheduler state.")

print("Model and optimizer states loaded.")

# Restore Training Variables
start_epoch = checkpoint.get('epoch', 0) # Get epoch completed
# If epoch is the completed one, start training from the next
resume_epoch = start_epoch # The loop will run from here

best_val_loss = checkpoint.get('val_loss', float('inf')) # Get the best loss achieved

# Initialize Early Stopping (match original parameters)
early_stopper = EarlyStopping(patience=patience_early_stop, delta=delta_early_stop)
# Set the early stopper's best loss to the loaded value
early_stopper.best_loss = best_val_loss

early_stopper.counter = checkpoint.get('early_stop_counter', 0)
early_stopper.early_stop = checkpoint.get('early_stop_flag', False) # Check if it already triggered


print(f"\n--- Resuming Training from Epoch {resume_epoch + 1} ---")
print(f"Previous best validation loss: {best_val_loss:.4f}")
if early_stopper.early_stop:
    print("Early stopping was triggered in the previous run. Exiting.")
    exit()



--- Loading State Dictionaries ---
Scheduler state not found in checkpoint, using new scheduler state.
Model and optimizer states loaded.
Initialized EarlyStopping: patience=3, delta=0.005

--- Resuming Training from Epoch 11 ---
Previous best validation loss: 2.4801


In [ ]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
print("\n--- Starting Resumed Training Loop ---")
for epoch in range(resume_epoch, num_epochs):
    current_epoch_number = epoch + 1
    start_time = time.time()
    epoch_train_loss = 0.0

    # Training Phase
    encoder.train()
    decoder.train()
    print(f"\nEpoch [{current_epoch_number}/{num_epochs}] - Training")
    for i, (images, captions) in enumerate(train_loader):
        images, captions = images.to(device), captions.to(device)
        optimizer.zero_grad()
        features = encoder(images)
        outputs = decoder(features, captions[:, :-1])
        loss = criterion(outputs.reshape(-1, vocab_size), captions[:, 1:].reshape(-1))
        loss.backward()
        optimizer.step()
        epoch_train_loss += loss.item()
        if (i + 1) % 100 == 0:
            print(f"Batch [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}")

    avg_train_loss = epoch_train_loss / len(train_loader)
    epoch_time = time.time() - start_time
    print(f"Epoch [{current_epoch_number}/{num_epochs}] Training completed in {epoch_time:.2f}s")
    print(f"Average Training Loss: {avg_train_loss:.4f}")

    # Validation Phase
    encoder.eval()
    decoder.eval()
    epoch_val_loss = 0.0
    print(f"Epoch [{current_epoch_number}/{num_epochs}] - Validation")
    with torch.no_grad():
        for val_images, val_captions in val_loader:
            val_images, val_captions = val_images.to(device), val_captions.to(device)
            val_features = encoder(val_images)
            val_outputs = decoder(val_features, val_captions[:, :-1])
            val_loss = criterion(val_outputs.reshape(-1, vocab_size), val_captions[:, 1:].reshape(-1))
            epoch_val_loss += val_loss.item()

    avg_val_loss = epoch_val_loss / len(val_loader)
    print(f"Epoch [{current_epoch_number}/{num_epochs}], Average Validation Loss: {avg_val_loss:.4f}")

    # Checkpoint Saving
    # Compare against the potentially updated best_val_loss
    if avg_val_loss < best_val_loss - delta_early_stop:
        print(f"Validation loss decreased ({best_val_loss:.4f} --> {avg_val_loss:.4f}). Saving model...")
        best_val_loss = avg_val_loss # Update best loss tracker
        save_dict = {
            'epoch': current_epoch_number, # Save the epoch number
            'encoder_state_dict': encoder.state_dict(),
            'decoder_state_dict': decoder.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'vocab': vocab,
            'val_loss': best_val_loss,
            # Save hyperparameters used for this run
            'embed_size': embed_size,
            'hidden_size': hidden_size,
            'num_layers': num_layers,
            'dropout_prob': dropout_prob,
            'fine_tune_encoder': fine_tune_encoder,
            'device': str(device), # Store device maybe
            'early_stop_counter': early_stopper.counter,
            'early_stop_flag': early_stopper.early_stop
        }
        torch.save(save_dict, checkpoint_path) # Overwrite the best model file
    else:
         print(f"Validation loss did not improve significantly from {best_val_loss:.4f}.")

    # Learning Rate Scheduling
    scheduler.step(avg_val_loss)

    # Early Stopping Check
    # Sync the early stopper's internal best loss before calling it
    early_stopper.best_loss = best_val_loss
    early_stopper(avg_val_loss)
    # Update best_val_loss in case early_stopper internally updates it (though current impl doesn't)
    best_val_loss = early_stopper.best_loss

    if early_stopper.early_stop:
        print("Early stopping criteria met. Stopping training.")
        break

print("\n--- Resumed Training Finished ---")
print(f"Best Validation Loss achieved: {best_val_loss:.4f}")


--- Starting Resumed Training Loop ---

Epoch [11/15] - Training
Batch [100/1294], Loss: 2.4579
Batch [200/1294], Loss: 2.5990
Batch [300/1294], Loss: 2.4190
Batch [400/1294], Loss: 2.5273
Batch [500/1294], Loss: 2.4743
Batch [600/1294], Loss: 2.8252
Batch [700/1294], Loss: 2.7027
Batch [800/1294], Loss: 2.5735
Batch [900/1294], Loss: 2.6313
Batch [1000/1294], Loss: 2.5925
Batch [1100/1294], Loss: 2.4983
Batch [1200/1294], Loss: 2.6223
Epoch [11/15] Training completed in 1182.73s
Average Training Loss: 2.6228
Epoch [11/15] - Validation
Epoch [11/15], Average Validation Loss: 2.5042
Validation loss did not improve significantly from 2.4801.
EarlyStopping: No significant improvement for 1/3 epochs.

Epoch [12/15] - Training
Batch [100/1294], Loss: 2.7597
Batch [200/1294], Loss: 2.5869
Batch [300/1294], Loss: 2.7050
Batch [400/1294], Loss: 2.6559
Batch [500/1294], Loss: 2.5117
Batch [600/1294], Loss: 2.4348
Batch [700/1294], Loss: 2.6611
Batch [800/1294], Loss: 2.5423
Batch [900/1294], L